Information 
- http://host.robots.ox.ac.uk/pascal/VOC/voc2007/ (dataset)
- https://github.com/keshik6/pascal-voc-classification/tree/master (github inspo)

## Dependencies 

In [1]:
from dataset import PascalVOC_Dataset

## Prepare Dataset

In [0]:
dataset_train = PascalVOC_Dataset(data_dir,
                                      year='2012', 
                                      image_set='train', 
                                      download=download_data, 
                                      transform=transformations, 
                                      target_transform=encode_labels)

Visualize Dataset

## Optimizer

## Training

## Testing 

## Quantitative Evaluation